imagenet事前学習済みのresnet50を分類器として用いて拡散モデルの

In [5]:
import torch
import torchvision.models as models

# 事前学習済みのResNet50をロード
resnet50 = models.resnet50(weights=models.ResNet50_Weights.IMAGENET1K_V1)
resnet50.eval()  # 推論モードに切り替え

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
resnet50 = resnet50.to(device)

# 画像を入力する場合は、前処理が必要です
from torchvision import transforms
from PIL import Image

preprocess = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    ),
])

# 例: 画像ファイルを読み込んで推論
# img = Image.open("your_image.jpg")
# input_tensor = preprocess(img).unsqueeze(0)  # バッチ次元を追加
# with torch.no_grad():
#     output = resnet50(input_tensor)
# print(output)

In [6]:
# ...既存のコード...

from torch.utils.data import DataLoader
from torchvision import datasets

# ImageNet-256のパス
data_dir = './imagenet-256'  # 必要に応じてパスを修正

# データセットとDataLoaderの作成
dataset = datasets.ImageFolder(root=data_dir, transform=preprocess)
dataloader = DataLoader(dataset, batch_size=32, shuffle=False, num_workers=2)

# 精度計算
correct = 0
total = 0

with torch.no_grad():
    for images, labels in dataloader:
        outputs = resnet50(images)
        _, predicted = outputs.max(1)
        correct += (predicted == labels).sum().item()
        total += labels.size(0)

accuracy = correct / total * 100
print(f'Accuracy on ImageNet-256: {accuracy:.2f}%')
# ...既存のコード...

RuntimeError: Input type (torch.FloatTensor) and weight type (torch.cuda.FloatTensor) should be the same or input should be a MKLDNN tensor and weight is a dense tensor